In [147]:
# Librerías.
import pandas as pd
import Funciones as f

In [ ]:
%%capture
%run "4. Categorias de candidatos.ipynb"

In [149]:
# Control.
# Guardar los Valores Originales antes de la Imputación (evitando duplicaciones y errores).
for Nombre_Dataframe, df in dfs_Finales.items():

    for Columna in df.columns:

        # Aplicar solo a columnas IP de Respuesta, no asociadas, y que NO sean ya copias originales.
        if (Columna.startswith('IP_Item_') and 
            '_Respuesta' in Columna and 
            '_Izq' not in Columna and 
            '_Der' not in Columna and 
            not Columna.endswith('_Original')):

            Columna_Original = f'{Columna}_Original'

            # Crear la columna solo si no existe aún.
            if Columna_Original not in df.columns:
                df[Columna_Original] = df[Columna]

In [150]:
# Ítems progresistas.
# [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]

# Variables progresistas.
Items_Progresistas = ['IP_Item_5_Respuesta', 'IP_Item_6_Respuesta', 'IP_Item_9_Respuesta', 'IP_Item_11_Respuesta', 'IP_Item_16_Respuesta', 'IP_Item_20_Respuesta', 'IP_Item_24_Respuesta', 'IP_Item_25_Respuesta', 'IP_Item_27_Respuesta', 'IP_Item_28_Respuesta']
Items_Progresistas_Tiempo = ['IP_Item_5_Tiempo', 'IP_Item_6_Tiempo', 'IP_Item_9_Tiempo', 'IP_Item_11_Tiempo', 'IP_Item_16_Tiempo', 'IP_Item_20_Tiempo', 'IP_Item_24_Tiempo', 'IP_Item_25_Tiempo', 'IP_Item_27_Tiempo', 'IP_Item_28_Tiempo']

In [151]:
# Ítems conservadores.
# [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]

Items_Conservadores = ['IP_Item_3_Respuesta', 'IP_Item_4_Respuesta', 'IP_Item_7_Respuesta', 'IP_Item_8_Respuesta', 'IP_Item_10_Respuesta', 'IP_Item_19_Respuesta', 'IP_Item_22_Respuesta', 'IP_Item_23_Respuesta', 'IP_Item_29_Respuesta', 'IP_Item_30_Respuesta']
Items_Conservadores_Tiempo = ['IP_Item_3_Tiempo', 'IP_Item_4_Tiempo', 'IP_Item_7_Tiempo', 'IP_Item_8_Tiempo', 'IP_Item_10_Tiempo', 'IP_Item_19_Tiempo', 'IP_Item_22_Tiempo', 'IP_Item_23_Tiempo', 'IP_Item_29_Tiempo', 'IP_Item_30_Tiempo']

In [152]:
# Combinar todas las columnas que necesitan procesamiento.
Items_Sin_Asociacion = (Items_Progresistas + Items_Progresistas_Tiempo + 
                       Items_Conservadores + Items_Conservadores_Tiempo)

In [153]:
# Convertir columnas de string a numérico.
for Nombre_df, df in dfs_Finales.items():
    for Columna in Items_Sin_Asociacion:
        if Columna in df.columns:
            df[Columna] = pd.to_numeric(df[Columna], errors='coerce')

In [ ]:
# Reporte de Medianas y Datos Faltantes por Columna.
for Nombre_df, df in dfs_Finales.items():

    print(f'\nANTES DEL RELLENADOR\n Reporte de Columnas IP para: {Nombre_df}\n')

    # Filtrar Columnas de Interés (Respuestas y Tiempos de IP Items no Asociados).
    Columnas = [Columna for Columna in df.columns if Columna.startswith('IP_Item_') and 
                (('_Respuesta' in Columna or '_Tiempo' in Columna) and 
                 '_Izq' not in Columna and '_Der' not in Columna)]

    for Columna in Columnas:
        Serie_Numerica = pd.to_numeric(df[Columna], errors='coerce')
        Faltantes = Serie_Numerica.isna().sum()

        print(f'{Columna:<35} | Faltantes: {Faltantes}')

In [155]:
# Rellenar valores vacíos en las columnas con la mediana de la población por Categoría PASO por cada df.
for Nombre_df, df in dfs_Finales.items():
    
    # Verificar que existe la columna de agrupación.
    if 'Categoria_PASO_2023' not in df.columns:
        print(f"  - Error: Columna 'Categoria_PASO_2023' no encontrada.")
        continue
    
    Columnas_Procesadas = 0
    
    # Procesar cada columna.
    for Columna in Items_Sin_Asociacion:
        if Columna in df.columns:
            # Rellenar valores faltantes con la mediana por grupo.
            df[Columna] = df.groupby('Categoria_PASO_2023')[Columna].transform(
                lambda Grupo: Grupo.fillna(Grupo.median())
            )
            Columnas_Procesadas += 1

In [ ]:
# Reporte de Medianas y Datos Faltantes por Columna.
for Nombre_df, df in dfs_Finales.items():

    print(f'\nDESPUÉS DEL RELLENADOR\n Reporte de Columnas IP para: {Nombre_df}\n')

    # Filtrar Columnas de Interés (Respuestas y Tiempos de IP Items no Asociados).
    Columnas = [Columna for Columna in df.columns if Columna.startswith('IP_Item_') and 
                (('_Respuesta' in Columna or '_Tiempo' in Columna) and 
                 '_Izq' not in Columna and '_Der' not in Columna)]

    for Columna in Columnas:
        Serie_Numerica = pd.to_numeric(df[Columna], errors='coerce')
        Faltantes = Serie_Numerica.isna().sum()

        print(f'{Columna:<35} | Faltantes: {Faltantes}')

In [ ]:
# Controlar que los Rellenos por Mediana coincidan con la Categoría Ideológica.
for Nombre_Dataframe, df in dfs_Finales.items():

    print(f'\nControl de Imputación por Categoría en: {Nombre_Dataframe}\n')

    Columnas = [Columna for Columna in df.columns if 
                Columna.startswith('IP_Item_') and
                '_Respuesta' in Columna and 
                '_Izq' not in Columna and 
                '_Der' not in Columna and
                not Columna.endswith('_Original')]

    for Columna in Columnas:
        Columna_Original = f'{Columna}_Original'
        
        # Verificar que la columna original existe.
        if Columna_Original not in df.columns:
            print(f"  - Advertencia: {Columna_Original} no encontrada.")
            continue
            
        Inconsistencias = 0

        for Categoria in df['Categoria_PASO_2023'].dropna().unique():
            df_Categoria = df[df['Categoria_PASO_2023'] == Categoria]
            Serie_Actual = pd.to_numeric(df_Categoria[Columna], errors='coerce')
            Serie_Original = pd.to_numeric(df_Categoria[Columna_Original], errors='coerce')
            Mediana_Categoria = Serie_Actual.median()

            Imputados_Invalidos = df_Categoria[
                Serie_Original.isna() & 
                (Serie_Actual != Mediana_Categoria)
            ]

            Inconsistencias += len(Imputados_Invalidos)

        print(f'{Columna:<35} | Rellenos Incorrectos: {Inconsistencias}')

In [158]:
# Exportar base para el control.
for Nombre_df, df in dfs_Finales.items():
    dfs_Finales[Nombre_df].head(50).to_excel(f'Controles/5. Relleno de medianas de IP no asociados ({Nombre_df}).xlsx', index=False)

In [159]:
# Exportar solo columnas imputadas para control
Items = [3, 4, 5, 6, 7, 8, 9, 10, 11, 16, 19, 20, 22, 23, 24, 25, 27, 28, 29, 30]
Columnas_Respuesta = [f'IP_Item_{i}_Respuesta' for i in Items]
Columnas_Tiempo = [f'IP_Item_{i}_Tiempo' for i in Items]

In [160]:
for Nombre_df, df in dfs_Finales.items():
    
    Columnas = ['ID', 'Orden_IP_Items'] + Columnas_Respuesta + Columnas_Tiempo
    Columnas_Presentes = [Columna for Columna in Columnas if Columna in df.columns]
    
    dfs_Finales[Nombre_df][Columnas_Presentes].head(50).to_excel(
        f'Controles/5. Relleno de medianas de IP no asociados ({Nombre_df}).xlsx',
        index=False
    )